In [1]:
import sympy as sp
import qutip as qt
import numpy as np
from tqdm import trange

from sympy.physics import quantum as spq
from sympy.physics.quantum.dagger import Dagger as spq_Dag

from scipy.signal import find_peaks
import matplotlib.pyplot as plt

(Ix, Iy, Iz) = qt.spin_J_set(1/2)
Ix2 = Ix**2
Iy2 = Iy**2
Iz2 = Iz**2
Im = Ix - 1j*Iy
Ip = Ix + 1j*Iy
I2 = Ix**2+Iy**2
Id = qt.qeye(2)
IdId = qt.tensor(Id, Id)
vals = np.round(Iz.eigenstates()[0], 1)
vects = -Iz.eigenstates()[1]
dw = vects[0]
up = vects[1]
upup = qt.tensor(up, up)
upup_dm = upup * upup.dag()

vals = np.round(Ix.eigenstates()[0], 1)
vects = -Ix.eigenstates()[1]
xp = -vects[1]
xm = vects[0]

vals = np.round(Iy.eigenstates()[0], 1)
vects = -Iy.eigenstates()[1]
yp = vects[1]
ym = vects[0]

def diag_exp(x):
    l = len(x)
    i = int(np.sqrt(l))
    for j in range(0, l, i+1):
        x[j] = sp.exp(sp.nsimplify(x[j]))
    return x
    
def pulserize(pulse_duration, rot_axis):
    return (-1j*pulse_duration*rot_axis).expm()

def sp_matrix(it):
    return sp.Matrix(np.array(it))

def round_expr(expr, num_digits):
    return expr.xreplace({n : round(n, num_digits) for n in expr.atoms(sp.Number)})

In [2]:
def create_symbol_dict(symbol, real, upper_limit=5):
    '''symbol : str
    '''
    x = {}
    for i in range(1, upper_limit):
        for j in range(1, upper_limit):
            if i==j:
                if real:
                    x[i, j] = sp.Symbol('{}{}{}'.format(symbol, i, j), real=True)
            else:
                x[i, j] = sp.Symbol('{}{}{}'.format(symbol, i, j), real=True)
                    
    return x

In [3]:
x = create_symbol_dict('x', True)
y = create_symbol_dict('y', False)
t, w_0, w_1, red_J = sp.symbols('t omega0 omega1 J', real=True)

In [4]:
matrix = []
for i in range(1, 5):
    row = []
    for j in range(1, 5):
        if i == j:
            row.append(x[i, j])
        else:
            row.append(x[i, j] + 1j * y[i, j])
    matrix.append(row)

In [5]:
D_rho = sp.Matrix(matrix)

In [6]:
D_rho

Matrix([
[            x11, x12 + 1.0*I*y12, x13 + 1.0*I*y13, x14 + 1.0*I*y14],
[x21 + 1.0*I*y21,             x22, x23 + 1.0*I*y23, x24 + 1.0*I*y24],
[x31 + 1.0*I*y31, x32 + 1.0*I*y32,             x33, x34 + 1.0*I*y34],
[x41 + 1.0*I*y41, x42 + 1.0*I*y42, x43 + 1.0*I*y43,             x44]])

In [7]:
#detection on both nuclei 
Detect = sp_matrix(qt.tensor(Ip, Id) + qt.tensor(Id, Ip))

#define the hamiltonain and the evolution operator
H_0 = w_0*sp_matrix(qt.tensor(Iz, Id)) + w_1*sp_matrix(qt.tensor(Id, Iz))
H_1 = 2*sp.pi*red_J*sp_matrix(qt.tensor(Iz, Iz))
U = sp.exp(-1j * t * H_0) * diag_exp(-1j * t * H_1)

In [8]:
rho = []
operations = [] 
ops_str = ['Id', 'Ix', 'Iy']
ops = [Id, Ix, Iy]
i = 0
for op1 in trange(len(ops)):
    R1 = sp_matrix(pulserize(-np.pi/2, qt.tensor(ops[op1], Id)))
    for op2 in range(len(ops)):
        R2 = sp_matrix(pulserize(-np.pi/2, qt.tensor(Id, ops[op2])))
        if op1 == op2 == 0:
            M = spq_Dag(U) * spq_Dag(D_rho) * U * Detect 
        else:
            M = spq_Dag(U) * spq_Dag(R2) * spq_Dag(R1) * D_rho * R1 * R2 * U * Detect
        rho.append(sp.simplify(sp.Trace(M)))
        operations.append('{}(x){}'.format(ops_str[op1], ops_str[op2]))

100%|███████████████████████████████████████████████████████████████████████████████████| 3/3 [05:45<00:00, 115.14s/it]


In [9]:
operations[0]

'Id(x)Id'

In [10]:
U * Detect 

Matrix([
[0, 1.0*exp(-0.5*I*t*(omega0 + omega1))*exp(-I*pi*J*t/2), 1.0*exp(-0.5*I*t*(omega0 + omega1))*exp(-I*pi*J*t/2),                                                   0],
[0,                                                    0,                                                    0, 1.0*exp(-0.5*I*t*(omega0 - omega1))*exp(I*pi*J*t/2)],
[0,                                                    0,                                                    0,  1.0*exp(0.5*I*t*(omega0 - omega1))*exp(I*pi*J*t/2)],
[0,                                                    0,                                                    0,                                                   0]])

In [11]:
rho[0]

1.0*(x12 - I*y12)*exp(I*t*(-pi*J - 1.0*omega1)) + 1.0*(x13 - I*y13)*exp(-I*t*(pi*J + 1.0*omega0)) + 1.0*((x24 - I*y24)*exp(I*t*(pi*J + 0.5*omega0 + 0.5*omega1)) + (x34 - I*y34)*exp(I*t*(pi*J + 1.5*omega0 - 0.5*omega1)))*exp(-I*t*(1.5*omega0 + 0.5*omega1))